In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
# Path to your Google Cloud service account credentials JSON file
credentials_location = '/home/hyderreza/.google/credentials/google_credentials.json'

# Create SparkConf object to configure Spark settings
conf = (
    SparkConf() 
    .setMaster('local[*]')  # Run Spark locally using all available CPU cores
    .setAppName('test')  # Name of the Spark application
    .set("spark.jars", "../lib/gcs-connector-hadoop3-2.2.5.jar")  # Add the GCS connector JAR
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")  # Enable service account authentication
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)  # Path to credentials
)

In [3]:
# Initialize SparkContext with the above configuration
sc = SparkContext(conf=conf)

# Get Hadoop configuration object to set file system-specific settings
hadoop_conf = sc._jsc.hadoopConfiguration()

# Define the abstract file system implementation for GCS
hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Define the actual GCS file system implementation class
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")

# Set the path to your credentials again for the Hadoop config (redundant but sometimes required)
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)

# Enable service account authentication for the Hadoop-level GCS configuration
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

your 131072x1 screen size is bogus. expect trouble
25/04/05 23:41:04 WARN Utils: Your hostname, Trydex resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/05 23:41:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/05 23:41:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Create a SparkSession using the same configuration as SparkContext
spark = (
    SparkSession.builder
    .config(conf=sc.getConf())  # Reuse the SparkConf from the SparkContext
    .getOrCreate()
)
spark

In [ ]:
# Now you can access buckets in google cloud storage via gs://
df_green = spark.read.parquet('gs://zoomcamp-454219-nytaxi/pq/green/*/*')
df_green.show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-11 04:05:54|  2020-01-11 04:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.